In [101]:
#%reset -f -s

In [102]:
'''------------------------------------------------------------------
---------------------------------------------------------------------
LSTM Training
---------------------------------------------------------------------
Version: V1.7       Modified: 06.11.2024        William Siegle
---------------------------------------------------------------------
notebook can be converted to python script using: 
(python -m) jupytext --to py FILENAME.ipynb
------------------------------------------------------------------'''
#import pathlib
from pathlib import Path, WindowsPath, PosixPath


# SETTINGS ------------------------------------------------------------------------
CONFIG = {
    # SYSTEM: ---------------------------------------------------------------------
    "GPU_SELECT":       1, # {0,1,2,3, None: CPU only}
    "ROOT":             Path('../..').resolve(),
    "INPUT_LOCATION":   Path("TripSequences", "trips_processed_pinn_2"), 
    "OUTPUT_LOCATION":  Path("src", "models", "pth"),
    "SEED"  :           100,
    "PLOT_ACTIVE":      True,

    # DATA PREPROCESSING: ---------------------------------------------------------
    "TRAIN_VAL_TEST":   [0.8, 0.19, 0.01], # [train, val, test splits]
    "MAX_FILES":        None, # None: all files
    "SCALERS":          {'feature_scaler': 'MaxAbsScaler()',
                        'target_scaler': 'MinMaxScaler(feature_range=(0, 1))',
                        'prior_scaler': 'MinMaxScaler(feature_range=(0, 1))'},
    "MIN_SEQ_LENGTH":   600, # minimum sequence length in s to be included in DataSets

    # FEATURES: -------------------------------------------------------------------
    "FEATURES":         ['actdrvtrnpwrprc_cval','altitude_cval_ippc','motortemperature_pti1',
                        'brktempra_cval','hv_batmaxdischrgpwrlim_cval_1',
                        'actualtorque_pti1','powerstagetemperature_pti1',
                        'accelpdlposn_cval','airtempoutsd_cval_cpc','airtempinsd_cval_hvac','actualdcvoltage_pti1','hv_batavcelltemp_cval_bms1',
                        'epto_pwr_cval','maxtracpwrpct_cval','airtempinsd_rq','bs_brk_cval','brc_stat_brc1','vehspd_cval_cpc',
                        'vehweight_cval_pt','actualspeed_pti1','selgr_rq_pt',
                        'rmsmotorcurrent_pti1','roadgrad_cval_pt','currpwr_contendrnbrkresist_cval','elcomp_pwrcons_cval',
                        'hv_ptc_cabin1_pwr_cval','maxrecuppwrprc_cval','txoiltemp_cval_tcm', 'start_soc'],
    "TARGETS":          ['hv_bat_soc_cval_bms1'],
    "PRIORS":           ['emot_soc_pred'],  

    # MODEL: -----------------------------------------------------------------------
    "HIDDEN_SIZE":      400,    # features in the hidden state h
    "NUM_LAYERS":       2,      # recurrent layers for stacked LSTMs. Default: 1
    "DROPOUT":          0.5,
    
    # TRAINING & OPTIMIZER: --------------------------------------------------------
    "NUM_EPOCHS":       10,
    "BATCH_SIZE":       16,   # [16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384, 32768]
    "LEARNING_RATE":    1e-3,   # 0.001 lr
    "OPTIMIZER":        "torch.optim.AdamW(model.parameters(), lr = LEARNING_RATE, weight_decay = 1e-3)",      
                        # weight_decay = 1e-4     # weight decay coefficient (default: 1e-2)
                        # betas = (0.9, 0.95),    # coefficients used for computing running averages of gradient and its square (default: (0.9, 0.999))
                        # eps = 1e-8,             # term added to the denominator to improve numerical stability (default: 1e-8)
    "LRSCHEDULER":      "torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience = 2, factor = 0.5, min_lr = 1e-7)",

    # LOSS FUNCTION: ---------------------------------------------------------------
    "CRITERION":        "nn.SmoothL1Loss()", # ['nn.MSELoss()', 'nn.L1Loss()', 'nn.SmoothL1Loss()', 'nn.HuberLoss()', 'MASE()']
    "LOSS_FN":          "F.mse_loss(output, target)", # ['F.mse_loss(output, target)', 'F.l1_loss(output, target)', 'F.smooth_l1_loss(output, target)', 'F.huber_loss(output, target)', 'F.mase_loss(output, target)']

    # SAVE & LOAD: -----------------------------------------------------------------
    "MODE":             "train_mode", # ['train_mode', 'test_mode']
    "TRAIN_LOG":        "test2.txt",

}
for key in CONFIG: globals()[key] = CONFIG[key];

___
LOCATE DEVICES & SYSTEM FOLDERS

In [103]:
# LOCATE REPOSITORY/DATASTORAGE IN CURRENT SYSTEM ENVIRONMENT  ---------------------------
# ----------------------------------------------------------------------------------------
import sys, os
for key in CONFIG: globals()[key] = CONFIG[key]
if 'ROOT' not in globals(): ROOT = Path('../..').resolve()
sys.path.append(os.path.abspath(ROOT))

# INTERNAL MODULE IMPORTS ----------------------------------------------------------------
# ----------------------------------------------------------------------------------------
from src.__init__ import *
from src.utils.data_utils import *
from src.utils.preprocess_utils import *
from src.utils.eval_utils import *
from src.utils.Trainers import *
#from src.LSTM.lstm_models import *

# SETUP ENVIRONMENT ---------------------------------------------------------------------
DATA_PATH, IS_NOTEBOOK, DEVICE = setup_environment(CONFIG, ROOT, SEED, GPU_SELECT)

------------------------------------------------------------
Directories:
  /home/sieglew/MA-eR-PINN:			project, ref, test, data, src, .git, archive
  /home/sieglew/MA-Data:			__pycache__, trips_processed_resampled, final, y_true, processed, TripSequences, trips_processed_pickles, final_2, trips_processed_final
------------------------------------------------------------
Running in notebook mode
CONFIG Dictionary:
---------------------------------------------------------------------------------------------------------------------------------
     Parameter        Value
--  ---------------  ------------------------------------------------------------------------------------------------------------
0   GPU_SELECT       1
1   ROOT             /home/sieglew/MA-eR-PINN
2   INPUT_LOCATION   TripSequences/trips_processed_pinn_2
3   OUTPUT_LOCATION  src/models/pth
4   SEED             100
5   PLOT_ACTIVE      True
6   TRAIN_VAL_TEST   [0.8, 0.19, 0.01]
7   MAX_FILES        None
8   SCALERS    

In [104]:
# FILE SOURCES ---------------------------------------------------------------
input_folder = Path(DATA_PATH, INPUT_LOCATION) # Trip parquet files
pth_folder = Path(ROOT, OUTPUT_LOCATION)

files, trip_lengths, indices_by_length, sorted_trip_lengths, all_signals \
    = prepare_data(input_folder, pth_folder, MAX_FILES, MIN_SEQ_LENGTH, ROOT)

------------------------------------------------------------
Total Files:	16116
Filtered Files:	12258
------------------------------------------------------------
                FileName  Length  Index
0        V13_T25.parquet   20843   1568
1       V18_T775.parquet   19425   4647
2       V13_T352.parquet   18308   1521
3       V18_T972.parquet   17858   6125
4      V16_T1629.parquet   17519   3101
...                  ...     ...    ...
12253   V12_T273.parquet     602  12093
12254   V16_T522.parquet     601   2517
12255  V18_T1549.parquet     601   5368
12256   V18_T883.parquet     601   7134
12257  V17_T3857.parquet     601   7143

[12258 rows x 3 columns]


___
DATA PREPROCESSING

In [105]:
# INPUT & TARGET SPECIFICATION ---------------------------------------------------
# these signals are required for the physical Model calculation:
base_signals = ["signal_time", "hirestotalvehdist_cval_icuc", "vehspd_cval_cpc", "altitude_cval_ippc", "airtempoutsd_cval_cpc", 
                "hv_batpwr_cval_bms1", "emot_pwr_cval","bs_roadincln_cval", "roadgrad_cval_pt"]

# these signals have to be dropped (from Features) in order for appropriate training:
columns_to_drop = ["signal_time",                       # works as index
                    "hirestotalvehdist_cval_icuc",      # starts from 0, obtained by speed integration
                    "latitude_cval_ippc",               # only GPS 
                    "longitude_cval_ippc",              # only GPS
                    "hv_batpwr_cval_bms1",              # directly related to target (soc_gradient)
                    "hv_batmomavldischrgen_cval_1",     # indirect target 1 in kWh
                    "hv_bat_soc_cval_bms1",              # indirect target 2 in %SoC
                    "soc_gradient",                     # actual target signal   
                    "emot_pwr_cval",                    # replaced as physical prior for PINN
                    "emot_pwr_pred",                    # actual physical prior for PINN
                    ]

# Ensure no element of "columns_to_drop" is included in "FEATURES"
assert not any(col in FEATURES for col in columns_to_drop), "Some columns to drop are still in FEATURES"

# ---------------------------------------------------
selection_1 = ["vehspd_cval_cpc", "altitude_cval_ippc", "airtempoutsd_cval_cpc", 'roadgrad_cval_pt', "vehweight_cval_pt", "accelpdlposn_cval", "bs_brk_cval", "elcomp_pwrcons_cval",
               "epto_pwr_cval", "motortemperature_pti1", "powerstagetemperature_pti1", 'airtempinsd_cval_hvac', 'brktempra_cval', 'selgr_rq_pt']
selection_2 = ["hirestotalvehdist_cval_icuc", "vehspd_cval_cpc", "altitude_cval_ippc", "airtempoutsd_cval_cpc", "hv_batpwr_cval_bms1", "emot_pwr_cval", "roadgrad_cval_pt"]
selection_3 = ["vehspd_cval_cpc", "altitude_cval_ippc", "airtempoutsd_cval_cpc", "vehweight_cval_pt"]


In [106]:
# FEATURE SELECTION  ----------------------------------------------------------------------------
INPUT_COLUMNS = FEATURES; TARGET_COLUMN = TARGETS; PRIOR_COLUMN = PRIORS
print(f"{'-'*60}\nInput Signals:\t{len(FEATURES)}\nTarget Signals:\t{len(TARGETS)}\nPhysical Prior Signals:\t{len(PRIORS)}\n{'-'*60}")

# FEATURE NORMALIZATION/SCALING -----------------------------------------------------------------
scaler = eval(SCALERS['feature_scaler'])
target_scaler = eval(SCALERS['target_scaler'])
prior_scaler = eval(SCALERS['prior_scaler'])

------------------------------------------------------------
Input Signals:	29
Target Signals:	1
Physical Prior Signals:	1
------------------------------------------------------------


In [107]:
INPUT_COLUMNS, TARGET_COLUMN, PRIOR_COLUMN

(['actdrvtrnpwrprc_cval',
  'altitude_cval_ippc',
  'motortemperature_pti1',
  'brktempra_cval',
  'hv_batmaxdischrgpwrlim_cval_1',
  'actualtorque_pti1',
  'powerstagetemperature_pti1',
  'accelpdlposn_cval',
  'airtempoutsd_cval_cpc',
  'airtempinsd_cval_hvac',
  'actualdcvoltage_pti1',
  'hv_batavcelltemp_cval_bms1',
  'epto_pwr_cval',
  'maxtracpwrpct_cval',
  'airtempinsd_rq',
  'bs_brk_cval',
  'brc_stat_brc1',
  'vehspd_cval_cpc',
  'vehweight_cval_pt',
  'actualspeed_pti1',
  'selgr_rq_pt',
  'rmsmotorcurrent_pti1',
  'roadgrad_cval_pt',
  'currpwr_contendrnbrkresist_cval',
  'elcomp_pwrcons_cval',
  'hv_ptc_cabin1_pwr_cval',
  'maxrecuppwrprc_cval',
  'txoiltemp_cval_tcm',
  'start_soc'],
 ['hv_bat_soc_cval_bms1'],
 ['emot_soc_pred'])

In [108]:
# GENERATE DATALOADERS ---------------------------------------------------------------

# DATA SET SPLITTING AND SORTING ----------------------------------------------------------------
train_subset, val_subset, test_subset = random_split(files, TRAIN_VAL_TEST)

# DATALOADER SETTINGS ------------------------------------------------------------------
dataloader_settings = {
    'batch_size': 1,                # see *Note above
    'shuffle': True,                # shuffle the batches before each epoch
    'collate_fn': collate_fn_PINN,       # include optional arguments
    'num_workers': 4,               # number of workers
    'pin_memory': False if DEVICE.type == 'cpu' else True
}

# PREPARE TRAIN, VAL & TEST DATALOADERS  ------------------------------------------------------------
train_subset, train_dataset, train_dataset_batches, train_loader = prepare_dataloader_PINN(train_subset, indices_by_length, \
    BATCH_SIZE, INPUT_COLUMNS, TARGET_COLUMN, PRIOR_COLUMN, scaler, target_scaler, prior_scaler, dataloader_settings, fit=True, drop_last=True)

val_subset, val_dataset, val_dataset_batches, val_loader = prepare_dataloader_PINN(val_subset, indices_by_length, \
    BATCH_SIZE, INPUT_COLUMNS, TARGET_COLUMN, PRIOR_COLUMN, scaler, target_scaler, prior_scaler, dataloader_settings)

test_subset, test_dataset, test_dataset_batches, test_loader = prepare_dataloader_PINN(test_subset, indices_by_length, \
    BATCH_SIZE, INPUT_COLUMNS, TARGET_COLUMN, PRIOR_COLUMN, scaler, target_scaler, prior_scaler, dataloader_settings)

'''
# PREPARE TRAIN, VAL & TEST DATALOADERS  ------------------------------------------------------------
train_subset, train_dataset, train_dataset_batches, train_loader = prepare_dataloader(train_subset, indices_by_length, \
    BATCH_SIZE, INPUT_COLUMNS, TARGET_COLUMN, scaler, target_scaler, dataloader_settings, fit=True)

val_subset, val_dataset, val_dataset_batches, val_loader = prepare_dataloader(val_subset, indices_by_length, \
    BATCH_SIZE, INPUT_COLUMNS, TARGET_COLUMN, scaler, target_scaler, dataloader_settings, fit=False)

test_subset, test_dataset, test_dataset_batches, test_loader = prepare_dataloader(test_subset, indices_by_length, \
    BATCH_SIZE, INPUT_COLUMNS, TARGET_COLUMN, scaler, target_scaler, dataloader_settings, fit=False)
    '''

# print dataset info
subset_files = print_dataset_sizes(train_dataset, val_dataset, test_dataset, train_subset, val_subset, test_subset, files)

 --> Warning: Removed the last 15 samples to ensure a balanced batch size
fitting Scalers: MaxAbsScaler, MinMaxScaler, MinMaxScaler
	50% of the fitting done...
Done. Create DataSets and DataLoaders...
	Number of batches created: 612
	Number of batches created: 146
	Number of batches created: 8
------------------------------------------------------------
Train size:  31979834		(Files: 9792)
Val. size:   7307710		(Files: 2329)
Test size:   425819		(Files: 122) 
 ------------------------------------------------------------
	Removed 15 file from the dataset
------------------------------------------------------------
first 3 train files: ['V18_T775.parquet', 'V13_T352.parquet', 'V18_T513.parquet']


In [109]:
if IS_NOTEBOOK and False: 
    check_batch_PINN(train_loader)
    #visualize_padding(BATCH_SIZE, trip_lengths, sorted_trip_lengths, train_loader, val_loader, test_loader)

In [110]:
'''
Move to model defintions
'''
# LSTM NETWORK -----------------------------------------------------------------------
class LSTM1_packed(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout, device=DEVICE):
        super(LSTM1_packed, self).__init__()

        self.input_size = input_size    # input size
        self.hidden_size = hidden_size  # hidden state
        self.num_layers = num_layers    # number of layers
        self.dropout = dropout
        self.device = device

        # LSTM CELL --------------------------------
        self.lstm = nn.LSTM(
            self.input_size,            # The number of expected features in the input x
            self.hidden_size,           # The number of features in the hidden state h
            self.num_layers,            # Number of recurrent layers for stacked LSTMs. Default: 1
            batch_first=True,           # If True, then the input and output tensors are provided as (batch, seq, feature) instead of (seq, batch, feature). Default: False
            bias=True,                  # If False, then the layer does not use bias weights b_ih and b_hh. Default: True
            dropout=self.dropout,       # usually: [0.2 - 0.5], introduces a Dropout layer on the outputs of each LSTM layer except the last layer, (dropout probability). Default: 0
            bidirectional=False,        # If True, becomes a bidirectional LSTM. Default: False
            proj_size=0,                # If > 0, will use LSTM with projections of corresponding size. Default: 0
            device=self.device
        )

        self.relu = nn.ReLU()
        self.fc_test = nn.Linear(hidden_size, 1)
    
    def forward(self, packed_input, hidden=None, cell=None):

        '''        
        # initial hidden and internal states (default to 0 if not explicitly provided)
        # --------------------------------
        h_0 = torch.zeros(self.num_layers, input.size(0) if batch_size is None else batch_size, self.hidden_size)
        c_0 = torch.zeros(self.num_layers, input.size(0) if batch_size is None else batch_size, self.hidden_size)  
        # --------------------------------
        out = self.relu(hn.view(-1, self.hidden_size)) # reshaping the data for Dense layer next
        out = self.fc_1(out) # first Dense
        out = self.relu(out) # relu
        out = self.fc(out) # Final Output
        '''

        # ALWAYS SET TO ZERO:::
        if hidden is None or cell is None:
            hidden = torch.zeros(self.num_layers, packed_input.batch_sizes[0].item(), self.hidden_size).to(self.device)
            cell = torch.zeros(self.num_layers, packed_input.batch_sizes[0].item(), self.hidden_size).to(self.device)

        # Propagate input through LSTM
        # --------------------------------
        # output, (hn, cn) = self.lstm(input, (h_0, c_0)) # lstm with input, hidden, and internal state
        # input shape:      (batch_size, seq_length, input_size)
        # output shape:     (batch_size, seq_length, hidden_size)

        packed_out, (hidden, cell) = self.lstm(packed_input, (hidden, cell))
        #print(f"LSTM: Output after LSTM: {packed_out.data.shape}, {type(packed_out)}")
        #print(f"             hidden: {hidden.shape}, {type(hidden)}")
        #print(f"             cell: {cell.shape}, {type(cell)}")

        # Unpack the output
        out, _ = pad_packed_sequence(packed_out, batch_first=True)
        #print(f"             after packing: {out.shape}, {type(out)}")

        # Output layers
        out = self.relu(out)  # relu
        #print(f"             after relu: {out.shape}, {type(out)}")

        out = self.fc_test(out)  # Use all outputs for prediction
        #print(f"             after fc: {out.shape}, {type(out)}")
        
        return out, (hidden, cell)
        

class LSTM1_packed_old_version(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, seq_length, dropout, device=DEVICE):
        super(LSTM1_packed_old_version, self).__init__()

        self.input_size = input_size    # input size
        self.hidden_size = hidden_size  # hidden state
        self.num_layers = num_layers    # number of layers
        self.dropout = dropout
        self.seq_length = seq_length    # sequence length

        # LSTM CELL --------------------------------
        self.lstm = nn.LSTM(
            self.input_size,            # The number of expected features in the input x
            self.hidden_size,           # The number of features in the hidden state h
            self.num_layers,            # Number of recurrent layers for stacked LSTMs. Default: 1
            batch_first=True,           # If True, then the input and output tensors are provided as (batch, seq, feature) instead of (seq, batch, feature). Default: False
            bias=True,                  # If False, then the layer does not use bias weights b_ih and b_hh. Default: True
            dropout=self.dropout,       # usually: [0.2 - 0.5], introduces a Dropout layer on the outputs of each LSTM layer except the last layer, (dropout probability). Default: 0
            bidirectional=False,        # If True, becomes a bidirectional LSTM. Default: False
            proj_size=0,                # If > 0, will use LSTM with projections of corresponding size. Default: 0
            device=device
        )

        self.relu = nn.ReLU()
        self.fc_test = nn.Linear(hidden_size, 1)
    
    def forward(self, packed_input, batch_size=None):
        # Propagate input through LSTM
        packed_out, _ = self.lstm(packed_input)
        #print(f"LSTM: Output after LSTM: {packed_out.data.shape}, {type(packed_out)}")

        # Unpack the output
        out, _ = pad_packed_sequence(packed_out, batch_first=True)
        #print(f"             after packing: {out.shape}, {type(out)}")

        # Output layers
        out = self.relu(out)  # relu
        #print(f"             after relu: {out.shape}, {type(out)}")

        out = self.fc_test(out)  # Use all outputs for prediction
        #print(f"             after fc: {out.shape}, {type(out)}")
        
        return out


In [111]:
'''def forward(self, packed_input, batch_size=None):

            
    # initial hidden and internal states
    # --------------------------------
    h_0 = torch.zeros(self.num_layers, input.size(0) if batch_size is None else batch_size, self.hidden_size)
    c_0 = torch.zeros(self.num_layers, input.size(0) if batch_size is None else batch_size, self.hidden_size)  
    # --------------------------------
    out = self.relu(hn.view(-1, self.hidden_size)) # reshaping the data for Dense layer next
    out = self.fc_1(out) # first Dense
    out = self.relu(out) # relu
    out = self.fc(out) # Final Output
    


    # Propagate input through LSTM
    packed_out, _ = self.lstm(packed_input)
    #print(f"LSTM: Output after LSTM: {packed_out.data.shape}, {type(packed_out)}")

    # Unpack the output
    out, _ = pad_packed_sequence(packed_out, batch_first=True)
    #print(f"             after packing: {out.shape}, {type(out)}")

    # Output layers
    out = self.relu(out)  # relu
    #print(f"             after relu: {out.shape}, {type(out)}")

    out = self.fc_test(out)  # Use all outputs for prediction
    #print(f"             after fc: {out.shape}, {type(out)}")
    
    return out'''

'def forward(self, packed_input, batch_size=None):\n\n            \n    # initial hidden and internal states\n    # --------------------------------\n    h_0 = torch.zeros(self.num_layers, input.size(0) if batch_size is None else batch_size, self.hidden_size)\n    c_0 = torch.zeros(self.num_layers, input.size(0) if batch_size is None else batch_size, self.hidden_size)  \n    # --------------------------------\n    out = self.relu(hn.view(-1, self.hidden_size)) # reshaping the data for Dense layer next\n    out = self.fc_1(out) # first Dense\n    out = self.relu(out) # relu\n    out = self.fc(out) # Final Output\n    \n\n\n    # Propagate input through LSTM\n    packed_out, _ = self.lstm(packed_input)\n    #print(f"LSTM: Output after LSTM: {packed_out.data.shape}, {type(packed_out)}")\n\n    # Unpack the output\n    out, _ = pad_packed_sequence(packed_out, batch_first=True)\n    #print(f"             after packing: {out.shape}, {type(out)}")\n\n    # Output layers\n    out = self.relu(o

In [113]:
# MODEL CONFIGURATION -----------------------------------------------------------------------

# LAYERS --------------------------------
input_size = len(INPUT_COLUMNS)     # expected features in the input x
hidden_size = HIDDEN_SIZE           # features in the hidden state h
num_layers = NUM_LAYERS             # recurrent layers for stacked LSTMs. Default: 1
num_classes = 1                     # output classes (=1 for regression)

# INSTANTIATE MODEL --------------------
model = LSTM1_packed_old_version(input_size, HIDDEN_SIZE, NUM_LAYERS, 60, DROPOUT).to(DEVICE)
print(f"{'-'*60}\n", model, f"{'-'*60}\nModel state_dict:")
for param_tensor in model.state_dict(): print(f"{param_tensor}:\t {model.state_dict()[param_tensor].size()}") 
# --> Note torch.Size([4*hidden_size, input_size]) for LSTM weights because of i,o,f,g params concatenated

------------------------------------------------------------
 LSTM1_packed_old_version(
  (lstm): LSTM(29, 400, num_layers=2, batch_first=True, dropout=0.5)
  (relu): ReLU()
  (fc_test): Linear(in_features=400, out_features=1, bias=True)
) ------------------------------------------------------------
Model state_dict:
lstm.weight_ih_l0:	 torch.Size([1600, 29])
lstm.weight_hh_l0:	 torch.Size([1600, 400])
lstm.bias_ih_l0:	 torch.Size([1600])
lstm.bias_hh_l0:	 torch.Size([1600])
lstm.weight_ih_l1:	 torch.Size([1600, 400])
lstm.weight_hh_l1:	 torch.Size([1600, 400])
lstm.bias_ih_l1:	 torch.Size([1600])
lstm.bias_hh_l1:	 torch.Size([1600])
fc_test.weight:	 torch.Size([1, 400])
fc_test.bias:	 torch.Size([1])


In [115]:
# TRAINING CONFIGURATION -----------------------------------------------------------------------

# OPTIMIZER, LR SCHEDULER, CRITERION----------------------------------------------------------------------------
optimizer = eval(OPTIMIZER) # common optimizers: ['torch.optim.Adam', 'torch.optim.SGD', 'torch.optim.RMSprop']
scheduler = eval(LRSCHEDULER)
criterion = eval(CRITERION) # criterion_list = [nn.MSELoss(), nn.L1Loss(), nn.SmoothL1Loss(), nn.HuberLoss(), MASE()]
print(f"{'-'*60}\n{optimizer}\n{'-'*60}\nLRScheduler: {scheduler.__class__}\n{'-'*60}\n{'-'*60}\nCriterion: {criterion.__class__}\n{'-'*60}\n")

------------------------------------------------------------
AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0.001
)
------------------------------------------------------------
LRScheduler: <class 'torch.optim.lr_scheduler.ReduceLROnPlateau'>
------------------------------------------------------------
------------------------------------------------------------
Criterion: <class 'torch.nn.modules.loss.SmoothL1Loss'>
------------------------------------------------------------



In [116]:
# LOSS FUNCTION ----------------------------------------------------------------   
def loss_fn_PINN_1(output, target, prior):
    loss = F.mse_loss(output, target, reduction='mean') # mean-squared error for regression
    return loss
# ---------------------------------------------------------------------------------
def loss_fn_PINN_2(output, target, prior):
    y_pred = output
    y_true = target
    y_phys = prior

    mse_loss = F.mse_loss(y_pred, y_true, reduction='mean')
    phys_loss = F.mse_loss(y_pred, y_phys, reduction='mean')

    total_loss = mse_loss + phys_loss
    return total_loss
# ---------------------------------------------------------------------------------
def loss_fn_PINN_3(output, target, prior):
    l_p = 0.8

    y_pred = output
    y_true = target
    y_phys = prior

    total_loss = F.mse_loss(y_true, (l_p * y_phys + (1 - l_p) * y_pred), reduction='mean')
    
    return total_loss

In [117]:
# SET NETWORK TRAINER -----------------------------------------------------------------
TRAINER = PTrainer_PINN(
    model = model, 
    optimizer = optimizer,
    scheduler = scheduler, 
    loss_fn = loss_fn_PINN_3, 
    train_loader = train_loader,
    val_loader = val_loader,
    test_loader = test_loader,
    num_epochs = NUM_EPOCHS, 
    device = DEVICE, 
    is_notebook = IS_NOTEBOOK,
    use_mixed_precision = False,
    log_file = TRAIN_LOG
    )

In [88]:
# START TRAINING -----------------------------------------------------------------
if MODE == 'train_mode': CHECKPOINT = TRAINER.train_model()

------------------------------------------------------------
Training Started.	Process ID: 431623 
------------------------------------------------------------
Model: LSTM1_packed_old_version	Parameters on device: CUDA:1
------------------------------------------------------------
Train/Batch size:	612 / 1
Loss:			<function loss_fn_PINN_3 at 0x7f460d9600e0>
Optimizer:		AdamW
LR:			0.001
Weight Decay:		0.001
------------------------------------------------------------


Epoch,Iteration,Batch Loss,Train Loss


  0%|          | 0/612 [00:00<?, ?batch/s]

In [20]:
CHECKPOINT, model_destination_path = save_checkpoint(TRAINER, train_loader, val_loader, test_loader, CHECKPOINT, CONFIG, subset_files, pth_folder)
'''
# COLLECTING RESULTS AND META DATA FOR SAVING DICT ---------------------------------------------------
trainer_add_info = {key: getattr(TRAINER, key) for key in ['model','optimizer','scheduler','state','clip_value','device','use_mixed_precision']}
loader_sizes = {'train_batches': len(train_loader), 'val_batches': len(val_loader), 'test_batches': len(test_loader)}
CHECKPOINT['CONFIG'] = CONFIG
CHECKPOINT = {**CHECKPOINT, **trainer_add_info, **subset_files, **loader_sizes}'''

In [ ]:
# SAVE CHECKPOINT ------------------------------------------------------------------------
# ----------------------------------------------------------------------------------------
# create unique identifier for model name
model_name_id = f'{model.__class__.__name__}_{datetime.now().strftime("%y%m%d_%H%M%S")}'
model_destination_path = Path(pth_folder, model_name_id + ".pth")

# SAVE (TORCH) -----------------------------------------------------------------
torch.save(CHECKPOINT, model_destination_path, pickle_protocol=pickle.HIGHEST_PROTOCOL)

# CHECK SAVED OBJECT SIZE -----------------------------------------------------------------
print(f"Model saved to:\t {model_destination_path}\n{'-'*60}\nSize: {os.path.getsize(model_destination_path) / 1024**2:.2f} MB\n{'-'*60}")
if os.path.getsize(model_destination_path) > 100 * 1024**2: 
    print("--> Warning: saved model size exceeds 100MB! Creating a zip file instead ...")
    try:
        import io, zipfile
        buffer = io.BytesIO(); torch.save(CHECKPOINT, buffer); buffer.seek(0)
        model_destination_path_zip = Path(pth_folder, model_name_id + ".zip")
        with zipfile.ZipFile(model_destination_path_zip, 'w', zipfile.ZIP_DEFLATED, compresslevel=9) as zipf: 
            zipf.writestr(model_destination_path_zip.name, buffer.read())
        if os.path.getsize(model_destination_path_zip) > 100 * 1024**2: print("--> Warning: zip compressed model size still exceeds 100MB!")
    except Exception as e: print(f"An error occurred while trying to save the model as a zip file: {e}")

___

In [ ]:
#%%skip

# LOAD CHECKPOINT -----------------------------------------------------------------
#model_destination_path = Path(pth_folder, "LSTM1_packed_241206_155418.pth")

# -----------------------------------------------------------------
try: 
    checkpoint = torch.load(model_destination_path, weights_only=False, \
        map_location=DEVICE if (torch.cuda.is_available() and GPU_SELECT is not None) else torch.device('cpu'))
except NotImplementedError:
    import pathlib
    temp = pathlib.PosixPath
    pathlib.PosixPath = pathlib.WindowsPath
    # temporarily change PosixPath to WindowsPath for loading
    checkpoint = torch.load(model_destination_path, weights_only=False, map_location=DEVICE if torch.cuda.is_available() else torch.device('cpu'))
    pathlib.PosixPath = temp

for key in checkpoint.keys(): globals()[key] = checkpoint[key]

# configure model and optimizer:
model.load_state_dict(model_state_dict)
optimizer.load_state_dict(optimizer_state_dict)

model.eval(); # set model to evaluation mode for inference
print(f"Model loaded from:\t{model_destination_path}\n{'-'*60}\nModel: {model.__class__.__name__}\tParameters on device: {next(model.parameters()).device}"
        f"\n{'-'*60}\nTrain/Batch size:\t{len(train_loader.dataset)} / {train_loader.batch_size}\n"
        f"Loss:\t\t\t{loss_fn}\nOptimizer:\t\t{optimizer.__class__.__name__}\nLR:\t\t\t"
        f"{optimizer.param_groups[0]['lr']}\nWeight Decay:\t\t{optimizer.param_groups[0]['weight_decay']}\n{'-'*60}\n", model)

In [ ]:
# PLOT TRAINING PERFORMANCE -----------------------------------------------------------------
# get DataFrame of training metrics:
training_df = pd.DataFrame(training_table, columns=["Epoch", "Iteration", "Batch Loss", "Train Loss"])

# -------------------------------------
NUM_EPOCHS = CONFIG['NUM_EPOCHS']
# plot training performance:
fig, ax1 = plt.subplots(figsize=(14,4))
ax1.set_xlabel('Epochs')
ax1.set_xticks(range(1, NUM_EPOCHS + 1))

assert len(train_losses_per_iter) == NUM_EPOCHS * train_batches, "Length of train_losses_per_iter might not match the number of iterations."
plt.plot(np.linspace(1, NUM_EPOCHS, len(train_losses_per_iter)), train_losses_per_iter, label='batch_loss', color='lightblue')
plt.plot(range(1, NUM_EPOCHS + 1), train_losses, label='train_loss', color='blue')
plt.plot(range(1, NUM_EPOCHS + 1), val_losses, label='val_loss', color='red')

plt.yscale('log'); fig.tight_layout(); plt.legend();

plt.text(0.86, 0.6, f"Train: {train_losses[-1]:.3e}\nVal:    {val_losses[-1]:.3e}", \
    transform=plt.gca().transAxes, fontsize=10, bbox=dict(facecolor='white', alpha=0.5));

if pd.Series(lr_history).nunique() > 1:
    ax2 = ax1.twinx()
    ax2.plot(range(1, NUM_EPOCHS + 1), lr_history, label='lr', color='green', linestyle='--')
    ax2.set_ylabel('Learning Rate', color='green'); ax2.tick_params(axis='y', labelcolor='green'); ax2.set_yscale('log')

In [ ]:
# EVALUATION -----------------------------------------------------------------

# get file list of test subset
test_files = checkpoint["test_files"]
# -------------------------------------
# evaluate model on test set
test_loss, outputs, targets, priors, original_lengths = TRAINER.evaluate_model()
# -------------------------------------
all_outputs, all_targets, all_original_lengths = [], [], []
for batch_outputs, batch_targets, batch_lengths in zip(outputs, targets, original_lengths):
    all_outputs.extend(batch_outputs)
    all_targets.extend(batch_targets)
    all_original_lengths.extend(batch_lengths)

# Inverse-transform on all outputs and targets for evaluation
scaled_outputs = [target_scaler.inverse_transform(output_sequence.reshape(1, -1)).squeeze() for output_sequence in all_outputs]
scaled_targets = [target_scaler.inverse_transform(target_sequence.reshape(1, -1)).squeeze() for target_sequence in all_targets]

print(f"Test Loss:  {test_loss:.6f}")
print(f"RMSE: {root_mean_squared_error(np.concatenate(scaled_targets), np.concatenate(scaled_outputs)):.4f}")
print(f"Standard Deviation: {np.std(np.concatenate(scaled_targets) - np.concatenate(scaled_outputs)):.4f}")

In [ ]:
# PLOT RESULTS -----------------------------------------------------------------
# get random sample sequence from test set
# -------------------------------------
sample_int = random.randint(1, len(scaled_outputs))
y_pred = scaled_outputs[sample_int]
y_true = scaled_targets[sample_int]

###############################################
# PLOT PREDICTION -----------------------------------------------------------------
if PLOT_ACTIVE:
     plt.figure(figsize=(18,4)); plt.xlabel('Time in s'); plt.ylabel('SOC in %'); plt.title('Battery State of Charge: Prediction vs. Actual Data') 
     plt.plot(y_true, label='Actual Data') # actual plot
     plt.plot(np.arange(0, len(y_true), 1), y_pred, label='Predicted Data') # predicted plot
     plt.legend()
     plt.text(0.01, 0.02, f"RMSE: {root_mean_squared_error(y_true, y_pred):.4f}\nStd Dev: {np.std(y_true - y_pred):.4f}",\
          transform=plt.gca().transAxes, fontsize=12, bbox=dict(facecolor='white', alpha=0.5))

     plt.figure(figsize=(18,4)); plt.xlabel('Time in s'); plt.ylabel('SOC in %')
     plt.plot(savgol_filter(y_true.flatten(), window_length=60, polyorder=3), label='Actual Data (Smoothed)') # actual plot
     plt.plot(np.arange(0, len(y_true), 1), savgol_filter(y_pred.flatten(), window_length=60, polyorder=3), label='Predicted Data (Smoothed)') # predicted plot
     plt.legend();

In [ ]:
'''
TODOs: 
- check if keeping the hidden state between batches is necessary
'''